# 01. Explanatory Data Analysis (EDA)

This notebook does EDA against the nvBench dataset and the preprocessed nvBench dataset in ncNet.
[The nvBench paper](https://arxiv.org/pdf/2112.12926.pdf) provides fundamental statistics against the dataset.
However, you can understand more if you check it yourself.

Firstly, we do EDA with pandas profiling against both datasets.
After that, we check them more deeply, especially for the data pandas-profiling can't handle well, like natural language.

As a result, we found several issues.

In the next notebook, we preprocess the dataset based on these EDA results.


## Setup


### Define Parameters

In [1]:
data_dir: str = "../data/"


### Load Modules

In [2]:
import json
import re
import sqlite3
import sys

from pathlib import Path
from random import Random

import altair as alt
import numpy as np
import pandas as pd

from IPython.display import display
from pandas_profiling import ProfileReport

from vxnli._vega_zero import VegaZero


In [3]:
DATA_DIR: Path = Path(data_dir)
DATASET_PATH: Path = DATA_DIR.joinpath("datasets/nvBench/NVBench.json")
DATABASE_DIR: Path = DATA_DIR.joinpath("datasets/nvBench/database/")
NCNET_DATASET_DIR: Path = DATA_DIR.joinpath("datasets/ncNet/dataset/")
PANDAS_PROFILING_DIR: Path = DATA_DIR.joinpath("pandas-profiling/")


In [4]:
PANDAS_PROFILING_DIR.mkdir(exist_ok=True)


In [5]:
# HACK: Add ncnet parser path (it doesn't support pip install)
sys.path.append(str(DATA_DIR.joinpath("datasets/ncNet/utilities").resolve()))

from vis_rendering import VegaZero2VegaLite


### Load Datasets

Load two datasets: the original nvBench dataset and the preprocessed dataset by the ncNet authors.

You need to provide at least two pieces of information to the model input: users' utterances and the database content.
The nl_queries column of nvBench contains the user's utterances.
The db_id column corresponds to the database, that is, sqlite3 files.

And the model output must contain the vis_query column information.
The vis_query.VQL column summarizes the other vis_query columns.

It is ok to predict the vis_query.VQL, however, it's better to preprocess them to make the model treat them well.
Fortunately, ncNet provides the VegaZero format, which the model can process efficiently.
You can check the VegaZero in the ncNet dataset.


In [6]:
with DATASET_PATH.open(mode="r") as f:
    df = json.load(f)

df = [{"id": k, **v} for k, v in df.items()]
df = pd.json_normalize(df)


In [7]:
df.head()


,id,chart,hardness,db_id,nl_queries,vis_query.vis_part,vis_query.data_part.sql_part,vis_query.data_part.binning,vis_query.VQL,vis_obj.chart,vis_obj.x_name,vis_obj.y_name,vis_obj.x_data,vis_obj.y_data,vis_obj.classify,vis_obj.describe
0,3,Scatter,Medium,activity_1,"[Show the faculty id of each faculty member, a...",Visualize SCATTER,"SELECT FacID , count(*) FROM Faculty AS T1 JOI...",,"Visualize SCATTER SELECT FacID , count(*) FROM...",scatter,FacID,count(*),"[[1121, 1148, 2192, 2311, 5718, 7134, 7271, 77...","[[3, 3, 4, 3, 2, 2, 2, 1, 1, 1, 1, 1, 3, 1, 1,...",[],
1,4,Pie,Medium,activity_1,[Show all the faculty ranks and the number of ...,Visualize PIE,"SELECT Rank , count(*) FROM Faculty AS T1 JOIN...",,"Visualize PIE SELECT Rank , count(*) FROM Facu...",pie,Rank,count(*),"[[AssocProf, AsstProf, Professor]]","[[2, 18, 14]]",[],
2,5,Bar,Medium,activity_1,[Show all the faculty ranks and the number of ...,Visualize BAR,"SELECT Rank , count(*) FROM Faculty AS T1 JOIN...",,"Visualize BAR SELECT Rank , count(*) FROM Facu...",bar,Rank,count(*),"[[AssocProf, AsstProf, Professor]]","[[2, 18, 14]]",[],
3,6,Pie,Easy,activity_1,[Show all the buildings along with the number ...,Visualize PIE,"SELECT Building , count(*) FROM Faculty GROUP ...",,"Visualize PIE SELECT Building , count(*) FROM ...",pie,Building,count(*),"[[Barton, Krieger, NEB]]","[[20, 20, 18]]",[],
4,7,Bar,Easy,activity_1,[Show all the buildings along with the number ...,Visualize BAR,"SELECT Building , count(*) FROM Faculty GROUP ...",,"Visualize BAR SELECT Building , count(*) FROM ...",bar,Building,count(*),"[[Barton, Krieger, NEB]]","[[20, 20, 18]]",[],


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7247 entries, 0 to 7246
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            7247 non-null   object
 1   chart                         7247 non-null   object
 2   hardness                      7247 non-null   object
 3   db_id                         7247 non-null   object
 4   nl_queries                    7247 non-null   object
 5   vis_query.vis_part            7247 non-null   object
 6   vis_query.data_part.sql_part  7247 non-null   object
 7   vis_query.data_part.binning   7247 non-null   object
 8   vis_query.VQL                 7247 non-null   object
 9   vis_obj.chart                 7247 non-null   object
 10  vis_obj.x_name                7247 non-null   object
 11  vis_obj.y_name                7247 non-null   object
 12  vis_obj.x_data                7247 non-null   object
 13  vis_obj.y_data    

In [9]:
df.describe()


,id,chart,hardness,db_id,nl_queries,vis_query.vis_part,vis_query.data_part.sql_part,vis_query.data_part.binning,vis_query.VQL,vis_obj.chart,vis_obj.x_name,vis_obj.y_name,vis_obj.x_data,vis_obj.y_data,vis_obj.classify,vis_obj.describe
count,7247,7247,7247,7247,7247,7247,7247,7247,7247,7247,7247,7247,7247,7247,7247,7247
unique,7247,7,4,152,7204,4,6086,128,6968,4,470,582,3150,3426,89,436
top,3,Bar,Medium,hr_1,[],Visualize BAR,"SELECT T2.Name , T1.Code FROM products AS T1 J...",,"Visualize BAR SELECT T2.Headquarter , T1.Code ...",bar,Name,count(*),"[[Mon, Tue, Wed, Thur, Fri, Sat, Sun]]","[[2, 1]]",[],
freq,1,5523,2802,945,5,5882,4,6059,4,5882,683,790,222,45,6687,2971


In [10]:
ncnet_train_df = pd.read_csv(NCNET_DATASET_DIR.joinpath("dataset_final/train.csv"))
ncnet_test_df = pd.read_csv(NCNET_DATASET_DIR.joinpath("dataset_final/test.csv"))
ncnet_val_df = pd.read_csv(NCNET_DATASET_DIR.joinpath("dataset_final/dev.csv"))

ncnet_train_df["subset"] = "train"
ncnet_test_df["subset"] = "test"
ncnet_val_df["subset"] = "val"

ncnet_df = pd.concat(
    [
        ncnet_train_df,
        ncnet_test_df,
        ncnet_val_df,
    ],
    ignore_index=True,
)

ncnet_df.head()


,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,subset
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,product_name,Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
1,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,product_name,Sony,mark bar data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
2,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,job age name,Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
3,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,job age name,Zach Bob Dan,mark bar data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
4,2545@y_name@DESC,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",Please give me a bar chart to show the average...,mark bar data pets encoding x pettype y aggreg...,pet_age petid pettype,cat,mark [T] data pets encoding x [X] y aggregate ...,<N> Please give me a bar chart to show the ave...,mark bar data pets encoding x pettype y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train


In [11]:
ncnet_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31588 entries, 0 to 31587
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tvBench_id         31588 non-null  object
 1   db_id              31588 non-null  object
 2   chart              31588 non-null  object
 3   hardness           31588 non-null  object
 4   query              31588 non-null  object
 5   question           31588 non-null  object
 6   vega_zero          31588 non-null  object
 7   mentioned_columns  29998 non-null  object
 8   mentioned_values   11782 non-null  object
 9   query_template     31588 non-null  object
 10  source             31588 non-null  object
 11  labels             31588 non-null  object
 12  token_types        31588 non-null  object
 13  subset             31588 non-null  object
dtypes: object(14)
memory usage: 3.4+ MB


In [12]:
ncnet_df.describe()


,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,subset
count,31588,31588,31588,31588,31588,31588,31588,29998,11782,31588,31588,31588,31588,31588
unique,3977,141,7,4,3906,15660,3709,1754,1482,1044,31400,3709,2378,3
top,1977@y_name@DESC,hr_1,Bar,Medium,"Visualize BAR SELECT bedType , avg(basePrice) ...",Show me about the proportion of Team_ID and Te...,mark bar data student encoding x city_code y a...,hire_date salary first_name last_name,Hall James King Banda Gee Lee Chen Bell Seo Ba...,mark bar data [D] encoding x [X] y aggregate [...,<N> Show me about the proportion of Team_ID an...,mark bar data student encoding x city_code y a...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl temp...,train
freq,18,4550,23264,13934,18,10,28,330,412,2607,5,28,478,25238


## Automatic Profiling with pandas-profiling

pandas-profiling is a python module which can generate profile reports automatically.
You need to process data manually for further investigation, but it is a good starting point.


In [13]:
ProfileReport(df).to_file(PANDAS_PROFILING_DIR.joinpath("nvbench.html"))


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/jupyter/vxnli/.venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
ProfileReport(ncnet_df).to_file(
    PANDAS_PROFILING_DIR.joinpath("ncnet-preprocessed-nvbench.html")
)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/jupyter/vxnli/.venv/lib/python3.9/site-packages/multimethod/__init__.py:315: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return func(*args, **kwargs)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## NL Queries

nl_queries in 20 rows contain non-ASCII characters, Chinese characters and apostrophes (‘ and ’).
In preprocessing, removing rows containing Chinese characters (or translating them into English) is essential.
It might be better to replace apostrophes with single/double quotes.

In [15]:
all_nl_queries = [q for qs in df["nl_queries"] for q in qs]

print(f"The total number of (flattened) nl_queries: {len(all_nl_queries)}")


The total number of (flattened) nl_queries: 25762


In [16]:
non_ascii_nl_queries = [q for q in all_nl_queries if not q.isascii()]

print(f"The number of non-ascii nl queries: {len(non_ascii_nl_queries)}")

has_non_ascii_nl_queries = df["nl_queries"].apply(
    lambda qs: any(not q.isascii() for q in qs)
)

print(f"The number of non-ascii rows: {len(df[has_non_ascii_nl_queries])}")

non_ascii_chars = (q for qs in non_ascii_nl_queries for q in qs)
non_ascii_chars = {c for txt in non_ascii_chars for c in txt if not c.isascii()}

print(f"Actual non-ascii characters: {non_ascii_chars}")

df[has_non_ascii_nl_queries].head()


The number of non-ascii nl queries: 32
The number of non-ascii rows: 20
Actual non-ascii characters: {'卡', '没', '息', '原', '据', '和', '数', '求', '始', '号', '‘', '义', '’', '信', '无', '有', '意'}


,id,chart,hardness,db_id,nl_queries,vis_query.vis_part,vis_query.data_part.sql_part,vis_query.data_part.binning,vis_query.VQL,vis_obj.chart,vis_obj.x_name,vis_obj.y_name,vis_obj.x_data,vis_obj.y_data,vis_obj.classify,vis_obj.describe
754,1048,Bar,Easy,customers_card_transactions,"[卡号求和无意义, What are card ids, customer ids, car...",Visualize BAR,"SELECT card_type_code , SUM(card_number) FROM ...",,"Visualize BAR SELECT card_type_code , SUM(card...",bar,card_type_code,SUM(card_number),"[[Credit, Debit]]","[[21491532119475148, 10744938002896200]]",[],GROUP BY card_type_code
1052,1386,Bar,Medium,farm,"[原始数据没有weekday信息, Give the years and official ...",Visualize BAR,"SELECT Year , COUNT(Year) FROM city AS T1 JOIN...",BIN Year BY WEEKDAY,"Visualize BAR SELECT Year , COUNT(Year) FROM c...",bar,Year,COUNT(Year),"[[Mon, Tue, Wed, Thur, Fri, Sat, Sun]]","[[0, 2, 1, 1, 0, 1, 1]]",[],BIN Year BY WEEKDAY
1543,2065,Pie,Medium,loan_1,[Find the name and account balance of the cust...,Visualize PIE,"SELECT cust_name , acc_bal FROM customer WHERE...",,"Visualize PIE SELECT cust_name , acc_bal FROM ...",pie,cust_name,acc_bal,"[[Mary, Jack]]","[[2000, 1000]]",[],
1544,2066,Bar,Medium,loan_1,[Find the name and account balance of the cust...,Visualize BAR,"SELECT cust_name , acc_bal FROM customer WHERE...",,"Visualize BAR SELECT cust_name , acc_bal FROM ...",bar,cust_name,acc_bal,"[[Mary, Jack]]","[[2000, 1000]]",[],
2054,2732,Pie,Extra Hard,scientist_1,[Find the number of the name of the project fo...,Visualize PIE,"SELECT T2.Name , COUNT(T2.Name) FROM assignedt...",,"Visualize PIE SELECT T2.Name , COUNT(T2.Name) ...",pie,Name,COUNT(Name),"[[Aerodynamics and Gas Mileage, Build Your Own...","[[1, 2]]",[],GROUP BY Name


## SQL

It is essential to parse SQLs to format them into a better representation for ML model.
However, some SQLs are broken.
Besides, sqlparse, the most famous SQL parsing module in python, can't parse some "non-broken" SQLs in this dataset well.


In [17]:
has_sql_error = np.zeros(len(df), dtype=bool)

for i, row in df.iterrows():
    con = row["db_id"]
    con = DATABASE_DIR.joinpath(f"{con}/{con}.sqlite")
    con = sqlite3.connect(con)

    cur = con.cursor()

    try:
        cur.execute(row["vis_query.data_part.sql_part"]).fetchall()
    except sqlite3.OperationalError:
        has_sql_error[i] = True
    finally:
        con.close()

print(f"the number of broken SQLs: {len(df[has_sql_error])}")

df[has_sql_error].head()


the number of broken SQLs: 145


,id,chart,hardness,db_id,nl_queries,vis_query.vis_part,vis_query.data_part.sql_part,vis_query.data_part.binning,vis_query.VQL,vis_obj.chart,vis_obj.x_name,vis_obj.y_name,vis_obj.x_data,vis_obj.y_data,vis_obj.classify,vis_obj.describe
130,158,Bar,Hard,baseball_1,[Show me the frequency of the weekdays (binnin...,Visualize BAR,"SELECT year , COUNT(year) FROM postseason AS T...",BIN year BY WEEKDAY,"Visualize BAR SELECT year , COUNT(year) FROM p...",bar,year,COUNT(year),"[[Mon, Tue, Wed, Thur, Fri, Sat, Sun]]","[[2, 3, 3, 2, 2, 1, 0]]",[],BIN year BY WEEKDAY
131,159,Bar,Hard,baseball_1,[Reutrn a bar chart to bin the year into day o...,Visualize BAR,"SELECT year , SUM(count(*)) FROM postseason AS...",BIN year BY WEEKDAY,"Visualize BAR SELECT year , SUM(count(*)) FROM...",bar,year,SUM(count(*)),"[[Mon, Tue, Wed, Thur, Fri, Sat, Sun]]","[[20, 25, 15, 20, 10, 5, 0]]",[],BIN year BY WEEKDAY
132,160,Bar,Hard,baseball_1,[Give me a bar chart that bins the year into d...,Visualize BAR,"SELECT year , AVG(count(*)) FROM postseason AS...",BIN year BY WEEKDAY,"Visualize BAR SELECT year , AVG(count(*)) FROM...",bar,year,AVG(count(*)),"[[Mon, Tue, Wed, Thur, Fri, Sat, Sun]]","[[10, 8.333333333333334, 5, 10, 5, 5, 0]]",[],BIN year BY WEEKDAY
133,161,Line,Hard,baseball_1,"[For each year, bin the year into day of the w...",Visualize LINE,"SELECT year , SUM(count(*)) FROM postseason AS...",BIN year BY YEAR,"Visualize LINE SELECT year , SUM(count(*)) FRO...",line,year,SUM(count(*)),"[[1903~1913, 1914~1924, 1925~1935, 1936~1946, ...","[[10, 15, 0, 0, 0, 0, 5, 5, 5, 40, 15]]",[],BIN year BY YEAR
134,162,Line,Hard,baseball_1,"[For each year, bin the year into day of the w...",Visualize LINE,"SELECT year , AVG(count(*)) FROM postseason AS...",BIN year BY YEAR,"Visualize LINE SELECT year , AVG(count(*)) FRO...",line,year,AVG(count(*)),"[[1903~1913, 1914~1924, 1925~1935, 1936~1946, ...","[[5, 5, 0, 0, 0, 0, 5, 5, 5, 10, 15]]",[],BIN year BY YEAR


In [18]:
ncnet_df.head()


,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,subset
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,product_name,Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
1,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,product_name,Sony,mark bar data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
2,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,job age name,Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
3,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,job age name,Zach Bob Dan,mark bar data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train
4,2545@y_name@DESC,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",Please give me a bar chart to show the average...,mark bar data pets encoding x pettype y aggreg...,pet_age petid pettype,cat,mark [T] data pets encoding x [X] y aggregate ...,<N> Please give me a bar chart to show the ave...,mark bar data pets encoding x pettype y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,train


In [19]:
tables = set()

for _, row in ncnet_df.iterrows():
    db_id = row["db_id"]

    vega_zero = row["vega_zero"]
    vega_zero = VegaZero.parse(vega_zero)

    table = vega_zero.data

    tables.add((db_id, table))

broken_tables = []

for db_id, table in tables:
    with sqlite3.connect(DATABASE_DIR.joinpath(f"{db_id}/{db_id}.sqlite")) as con:
        try:
            pd.read_sql(f"SELECT * FROM {table}", con)
        except:
            broken_tables.append((db_id, table))

broken_tables


[('wta_1', 'players')]

## ncNet Preprocessed Dataset

ncNet seems to use a subset of the nvBench dataset.
For instance, it lacks rows which contains SQL join operation.
Instead, it has the vega_zero column, which has the same information as the query column but is easier to handle for ML models.

However, there are several concerns or bugs in vega_zero.
Some are originated from nvBench, and the others are from VegaZero itself.


In [20]:
# ncNet has a data leakage among database.
# That is, ncNet model already knows the content of databases before evaluating it on test dataset.
# It is ok if you don't use the other databases, but it is not good if we thin k the real world usage.
set(ncnet_train_df["db_id"].unique()) & set(ncnet_test_df["db_id"].unique())


{'activity_1',
 'allergy_1',
 'apartment_rentals',
 'baseball_1',
 'behavior_monitoring',
 'bike_1',
 'body_builder',
 'candidate_poll',
 'cinema',
 'climbing',
 'coffee_shop',
 'college_1',
 'college_2',
 'college_3',
 'company_1',
 'company_office',
 'concert_singer',
 'cre_Doc_Tracking_DB',
 'cre_Docs_and_Epenses',
 'cre_Drama_Workshop_Groups',
 'cre_Theme_park',
 'culture_company',
 'customer_complaints',
 'customers_and_invoices',
 'customers_campaigns_ecommerce',
 'customers_card_transactions',
 'department_store',
 'device',
 'document_management',
 'dog_kennels',
 'dorm_1',
 'driving_school',
 'e_learning',
 'election',
 'employee_hire_evaluation',
 'entrepreneur',
 'farm',
 'flight_1',
 'game_1',
 'gas_company',
 'hr_1',
 'inn_1',
 'insurance_policies',
 'loan_1',
 'local_govt_and_lot',
 'machine_repair',
 'manufactory_1',
 'match_season',
 'mountain_photos',
 'music_4',
 'news_report',
 'orchestra',
 'party_people',
 'perpetrator',
 'phone_market',
 'product_catalog',
 'produ

In [21]:
# The number of rows are smaller than the original nvBench dataset
ncnet_unique_df = ncnet_df[["db_id", "chart", "hardness", "query", "vega_zero"]]
ncnet_unique_df = ncnet_unique_df.drop_duplicates()
ncnet_unique_df = ncnet_unique_df.reset_index()
ncnet_unique_df


,index,db_id,chart,hardness,query,vega_zero
0,0,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",mark bar data products encoding x product_name...
1,2,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",mark bar data person encoding x job y aggregat...
2,4,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",mark bar data pets encoding x pettype y aggreg...
3,6,products_for_hire,Bar,Extra Hard,"Visualize BAR SELECT payment_date , COUNT(paym...",mark bar data payments encoding x payment_date...
4,8,election,Bar,Easy,"Visualize BAR SELECT County_name , Population ...",mark bar data county encoding x county_name y ...
...,...,...,...,...,...,...
3916,31248,flight_company,Bar,Medium,"Visualize BAR SELECT Country , count(*) FROM a...",mark bar data airport encoding x country y agg...
3917,31300,flight_company,Bar,Medium,"Visualize BAR SELECT Country , count(*) FROM a...",mark bar data airport encoding x country y agg...
3918,31334,flight_company,Bar,Medium,"Visualize BAR SELECT Country , count(*) FROM a...",mark bar data airport encoding x country y agg...
3919,31486,restaurant_1,Bar,Medium,"Visualize BAR SELECT Fname , COUNT(Fname) FROM...",mark bar data student encoding x fname y aggre...


In [22]:
# No queries which contains join operation
ncnet_unique_df[ncnet_unique_df["query"].str.contains(" join ", flags=re.IGNORECASE)]


,index,db_id,chart,hardness,query,vega_zero


In [23]:
# Some SQLs are broken

ncnet_unique_has_sql_error = np.zeros(len(ncnet_unique_df), dtype=bool)

for i, row in ncnet_unique_df.iterrows():
    con = row["db_id"]
    con = DATABASE_DIR.joinpath(f"{con}/{con}.sqlite")
    con = sqlite3.connect(con)

    cur = con.cursor()

    # Remove VISUALIZE CHART_TYPE
    [_, _, query] = row["query"].split(" ", maxsplit=2)

    # Remove BIN ...
    [query, *_] = query.rsplit("BIN", maxsplit=1)

    try:
        cur.execute(query).fetchall()
    except sqlite3.OperationalError:
        ncnet_unique_has_sql_error[i] = True
    finally:
        con.close()

ncnet_unique_df[ncnet_unique_has_sql_error]


,index,db_id,chart,hardness,query,vega_zero
1166,2836,college_1,Scatter,Easy,"Visualize SCATTER SELECT max(stu_gpa) , min(st...",mark point data student encoding x max(stu_gpa...
2205,7280,department_store,Scatter,Easy,"Visualize SCATTER SELECT max(product_price) , ...",mark point data products encoding x max(produc...
2293,7754,wine_1,Scatter,Easy,"Visualize SCATTER SELECT max(Price) , max(Scor...",mark point data wine encoding x max(price) y a...
2432,8722,manufactory_1,Bar,Medium,"Visualize BAR SELECT Name , Price FROM product...",mark bar data products encoding x name y aggre...
2774,12426,game_1,Scatter,Easy,"Visualize SCATTER SELECT min(age) , max(age) F...",mark point data student encoding x min(age) y ...
2956,15770,wine_1,Scatter,Medium,"Visualize SCATTER SELECT avg(Price) , avg(Scor...",mark point data wine encoding x avg(price) y a...
3191,25388,gas_company,Bar,Medium,"Visualize BAR SELECT Company , Market_Value FR...",mark bar data company encoding x company y agg...
3424,26074,manufactory_1,Pie,Medium,"Visualize PIE SELECT Name , Price FROM product...",mark arc data products encoding x name y aggre...
3732,29664,department_store,Scatter,Easy,"Visualize SCATTER SELECT max(product_price) , ...",mark point data products encoding x max(produc...


In [24]:
# ncNet still has non-ascii question
len([q for q in ncnet_df["question"].unique() if not q.isascii()])


11

In [25]:
# ncNet vega_zero has only lower case characters
all(ncnet_df["vega_zero"] == ncnet_df["vega_zero"].str.lower())


True

In [26]:
# vega_zero filter might contain both ' or "
# however, it doesn't contain both in the same example
ncnet_df[
    ncnet_df["vega_zero"].str.contains('"') & ncnet_df["vega_zero"].str.contains("'")
]


,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,subset


In [27]:
# vega_zero sometimes contain "," mistakenly due to a ncNet bug
len(ncnet_df[ncnet_df["vega_zero"].str.contains(",")])


40

In [28]:
rand = Random(123)

# However, it seems to be no problem if you remove ','
for i in rand.sample(range(0, 40), 5):
    tmp = ncnet_df[ncnet_df["vega_zero"].str.contains(",")].iloc[i]

    print(tmp["query"])
    print(tmp["vega_zero"])
    print()


Visualize BAR SELECT DISTINCT state , COUNT(state) FROM votes GROUP BY state
mark bar data votes encoding x distinct y aggregate , count state transform group state

Visualize BAR SELECT DISTINCT state , COUNT(state) FROM votes GROUP BY state
mark bar data votes encoding x distinct y aggregate , count state transform group state

Visualize BAR SELECT DISTINCT state , COUNT(state) FROM votes GROUP BY state ORDER BY state DESC
mark bar data votes encoding x distinct y aggregate , count state transform group state sort state desc

Visualize BAR SELECT DISTINCT state , COUNT(state) FROM votes GROUP BY state ORDER BY state ASC
mark bar data votes encoding x distinct y aggregate , count state transform group state sort state asc

Visualize PIE SELECT DISTINCT state , COUNT(state) FROM votes GROUP BY state
mark arc data votes encoding x distinct y aggregate , count state transform group state



In [29]:
# "Extra Hard" examples have only bar & line charts
# This can be a problem to choose tasks for user study
ncnet_df[ncnet_df["hardness"] == "Extra Hard"]["chart"].value_counts()


Bar              1962
Stacked Bar       430
Grouping Line     214
Line              152
Name: chart, dtype: int64

## VegaZero Parser

The ncNet authors provide a parser for the VegaZero format, however, it has several bugs.

https://github.com/Thanksyy/ncNet/blob/19e852368228ad251a28623950524a364ee95260/utilities/vis_rendering.py

Some are originated from the nvBench dataset, but some are from its implementation.
We decided to re-implement (refactor) it and solve half of them, but there are still some issues.


In [30]:
# The ncNet parser contains the bugs below, but our parser handle these issues well
RESOLVED_BUG_EXAMPLES = [
    # a pie chart which has a filter
    {
        "db_id": "department_store",
        "table": "products",
        "sql": "SELECT product_type_code , min(product_price) FROM products GROUP BY product_type_code ORDER BY product_type_code",
        "vega_zero": "mark arc encoding x product_type_code y aggregate min product_price transform group x sort x asc",
    },
    # a bar chart which sorts x-axis in the reversed alphabet order
    {
        "db_id": "hr_1",
        "table": "employees",
        "sql": "SELECT JOB_ID , SUM(SALARY) FROM employees WHERE first_name LIKE '%D%' OR first_name LIKE '%S%' GROUP BY JOB_ID ORDER BY JOB_ID DESC",
        "vega_zero": 'mark bar encoding x job_id y aggregate sum salary transform filter first_name like "%d%" or first_name like "%s%" group x sort x desc',
    },
    # = operator
    {
        "db_id": "inn_1",
        "table": "rooms",
        "sql": 'SELECT decor , count(*) FROM Rooms WHERE bedType = "King" GROUP BY decor ORDER BY decor DESC',
        "vega_zero": 'mark bar encoding x decor y aggregate count decor transform filter bedtype = "king" group x sort x desc',
    },
    # like operator in a pie chart
    {
        "db_id": "loan_1",
        "table": "customer",
        "sql": "SELECT cust_name , acc_bal FROM customer WHERE cust_name LIKE '%a%'",
        "vega_zero": 'mark arc encoding x cust_name y aggregate none acc_bal transform filter cust_name like "%a%"',
    },
    # not like operator
    {
        "db_id": "hr_1",
        "table": "employees",
        "sql": "SELECT HIRE_DATE , AVG(SALARY) FROM employees WHERE first_name NOT LIKE '%M%'  ORDER BY AVG(SALARY) DESC ",
        "vega_zero": 'mark bar encoding x hire_date y aggregate mean salary transform filter first_name not like "%m%" sort y desc bin x by weekday',
    },
    # filter when timeUnit used
    {
        "db_id": "hr_1",
        "table": "employees",
        "sql": "SELECT HIRE_DATE , AVG(SALARY) FROM employees WHERE hire_date < '2002-06-21' ",
        "vega_zero": 'mark bar encoding x hire_date y aggregate mean salary transform filter hire_date < "2002-06-21" bin x by month',
    },
]


In [31]:
# These bugs below are not resolved yet... (there might be more not-found issues)
UNRESOLVED_BUG_EXAMPLES = [
    # The output figure cannot be sorted when timeUnit is used
    {
        "db_id": "dog_kennels",
        "table": "dogs",
        "sql": "SELECT date_departed , COUNT(date_departed) FROM Dogs  ORDER BY COUNT(date_departed) DESC ",
        "vega_zero": "mark bar encoding x date_departed y aggregate count date_departed transform sort y desc bin x by day",
    },
    # distinct operation
    {
        "db_id": "college_1",
        "table": "department",
        "sql": "SELECT SCHOOL_CODE , count(DISTINCT dept_address) FROM department",
        "vega_zero": "mark bar encoding x school_code y aggregate count distinct dept_address",
    },
    # group by other columns but x or y
    {
        "db_id": "manufactory_1",
        "table": "manufacturers",
        "sql": "SELECT Name , max(revenue) FROM manufacturers GROUP BY Headquarter ORDER BY max(revenue) DESC",
        "vega_zero": "mark bar encoding x name y aggregate max revenue transform group headquarter sort y desc",
    },
    # calculation in select
    {
        "db_id": "hr_1",
        "table": "jobs",
        "sql": "SELECT JOB_TITLE , max_salary - min_salary FROM jobs WHERE max_salary BETWEEN 12000 AND 18000",
        "vega_zero": "mark bar encoding x job_title y aggregate none max_salary - min_salary transform filter max_salary between 12000 and 18000",
    },
    # use aggregation against x axis
    {
        "db_id": "world_1",
        "table": "country",
        "sql": "SELECT sum(Population) , avg(LifeExpectancy) FROM country GROUP BY Continent",
        "vega_zero": "mark point encoding x sum(population) y aggregate mean lifeexpectancy transform group continent",
    },
    # sort by other columns but x or y
    {
        "db_id": "college_3",
        "table": "course",
        "sql": "SELECT Days , COUNT(Days) FROM COURSE GROUP BY Days ORDER BY Credits",
        "vega_zero": "mark bar encoding x days y aggregate count days transform group x sort credits asc",
    },
    # group by other columns but x or y
    {
        "db_id": "university_basketball",
        "table": "basketball_match",
        "sql": "SELECT Team_ID , All_Games_Percent FROM basketball_match GROUP BY Team_Name",
        "vega_zero": "mark point encoding x team_id y aggregate none all_games_percent transform group team_name",
    },
]


In [32]:
def execute_sql(db_id: str, sql: str) -> pd.DataFrame:
    with sqlite3.connect(DATABASE_DIR.joinpath(f"{db_id}/{db_id}.sqlite")) as con:
        return pd.read_sql(sql, con)


def load_table(db_id: str, table: str) -> pd.DataFrame:
    return execute_sql(db_id, f"SELECT * FROM {table}")


def preprocess_table(df: pd.DataFrame) -> pd.DataFrame:
    # As we see above, vega_zero (in ncnet) is lower-cased
    df = df.rename(columns={col: col.lower() for col in df.columns})

    for col_name, col_dtype in zip(df.columns, df.dtypes):
        if pd.api.types.is_string_dtype(col_dtype):
            df[col_name] = df[col_name].str.lower()

    return df


def compare_vega_zero_parsers(db_id: str, table: str, sql: str, vega_zero: str):
    df = load_table(db_id, table)
    df = preprocess_table(df)

    print("Table:")
    display(df.head(n=5))

    print("SQL Result:")

    display(execute_sql(db_id, sql).head(n=5))

    vega_zero = VegaZero.parse(vega_zero)

    # Restore data field for the ncNet vega-zero parser (removed in preprocessing because it's no need)
    vega_zero.data = table

    try:
        ncnet_vega_lite = VegaZero2VegaLite().to_VegaLite(str(vega_zero), df)
        ncnet_vega_lite_w_o_data = {
            k: v for k, v in ncnet_vega_lite.items() if k != "data"
        }
    except Exception as e:
        ncnet_vega_lite = None
        ncnet_vega_lite_w_o_data = None

    try:
        our_vega_lite = vega_zero.to_vega_lite(df)
        our_vega_lite_w_o_data = {k: v for k, v in our_vega_lite.items() if k != "data"}
    except Exception as e:
        our_vega_lite = None
        our_vega_lite_w_o_data = None

    if (
        str(ncnet_vega_lite_w_o_data) == str(our_vega_lite_w_o_data)
        and ncnet_vega_lite is not None
        and our_vega_lite is not None
    ):
        print("Vega-Lite: ", our_vega_lite_w_o_data)

        display(alt.Chart.from_dict(our_vega_lite))
    else:
        if ncnet_vega_lite is None:
            print("Vega-Lite (ncNet): ERROR")
        else:
            print("Vega-Lite (ncNet):", ncnet_vega_lite_w_o_data)

            display(alt.Chart.from_dict(ncnet_vega_lite))

        if our_vega_lite is None:
            print("Vega-Lite (ours): ERROR")
        else:
            print("Vega-Lite (ours): ", our_vega_lite_w_o_data)

            display(alt.Chart.from_dict(our_vega_lite))


In [33]:
RESOLVED_BUG_EXAMPLES[0]


{'db_id': 'department_store',
 'table': 'products',
 'sql': 'SELECT product_type_code , min(product_price) FROM products GROUP BY product_type_code ORDER BY product_type_code',
 'vega_zero': 'mark arc encoding x product_type_code y aggregate min product_price transform group x sort x asc'}

In [34]:
compare_vega_zero_parsers(**RESOLVED_BUG_EXAMPLES[0])


Table:


,product_id,product_type_code,product_name,product_price
0,1,clothes,red jeans,734.73
1,2,clothes,yellow jeans,687.23
2,3,clothes,black jeans,695.16
3,4,clothes,blue jeans,939.57
4,5,clothes,red jeans,534.52


SQL Result:


,product_type_code,min(product_price)
0,Clothes,408.82
1,Hardware,612.46


Vega-Lite (ncNet): ERROR
Vega-Lite (ours):  {'mark': 'arc', 'encoding': {'color': {'field': 'product_type_code', 'type': 'nominal'}, 'theta': {'field': 'product_price', 'type': 'quantitative', 'aggregate': 'min'}, 'order': {'field': 'value', 'type': 'quantitative', 'sort': 'ascending'}}}


alt.Chart(...)